In [1]:
import numpy as np
from astropy.io import fits
import nbimporter
from model_calls import showim, mkbias, mkdark, mkrdns
from os.path import splitext

Importing Jupyter notebook from model_calls.ipynb


In [7]:
texpo = 5.0*60.0  # exposure time in s
mulmd = 0.5    # multiplier modifier due to zemax sequential artifacts
nllvl = 1.0    # non-linearity level in percent
darkl = 1.0      # dark current in e/px/hr
queff = 0.90     # quantum efficiency in max-1
ronoi = 4.0      # readout noise in e
fwell = 350000  # full well capacity in e
ampst = 7.0   # amplifier sensitivity in uV/e
pxsiz = 15.0    # pixel size in um/px
resol = 4096  # resolution in pixels
ctrbt = 16  # controller bits
nbias = 10  # number of bias frames to be saved
ndark = 10  # number of dark frames to be saved
shima = False  # whether to show image in plotly
wpfit = False  # whether to write fits files of photons

In [8]:
filenm = 'TXT/sdss-r.txt'  # filename

In [9]:
maxv = fwell*ampst   # maximum amplifier voltage in uV
minv = 0.0    # minimum amplifier voltage in uV
maxc = (2**ctrbt)-1  # maximum counts
vtcn = maxc/(maxv-minv)    # uV to counts conversion factor in ADU/uV

In [11]:
multiread = np.genfromtxt(
    fname=(splitext(filenm)[0] + ('.unit')), dtype=float, skip_header=1)   # read the unit file's 2nd line
multip = multiread[-1]    # last entry is the multiplier
multip *= mulmd
print('Photon Multiplier: %.2f\nZemax Correction: %.2f\nTotal: %.2f' %
      (multiread[-1], mulmd, multip))

Photon Multiplier: 58.20
Zemax Correction: 0.50
Total: 29.10


In [13]:
# read file, image is now in photons unmultiplied
image = np.genfromtxt(fname=filenm, dtype=float, skip_header=17)
# multiply by the multiplier, image is now in ph/mm^2
image = np.multiply(image, multip)
# convert to ph/px^2
image = np.multiply(image, np.power((pxsiz*0.001), 2.0))
print('Pixel Area mm:%.2e' % np.power((pxsiz*0.001), 2.0))

Pixel Area mm:2.25e-04


In [7]:
if shima:
    showim(image)  # show image in plotly

In [8]:
if wpfit:
    hdu = fits.PrimaryHDU(image)   # make an HDU for the photon image
    # write in the header that the image is in ph
    hdu.header['UNIT'] = ('ph', 'Pixel intensity units')
    hdu.writeto(splitext(filenm)[0] + ('_ph.fit'))   # write the fits

In [9]:
biaslist = []    # list to be filled with bias frames in ADU/px^2
for i in range(0, nbias):
    biaslist.append(mkbias(maxc, nllvl, resol))
biasfr = mkbias(maxc=maxc, nllvl=nllvl, resol=resol)   # bias frame in ADU/px^2

In [10]:
darklist = []      # list to be filled with dark frames in e/px^2
for i in range(0, ndark):
    darklist.append(mkdark(darkl, texpo, resol))
# dark frame in e/px^2
darkfr = mkdark(darkl=darkl, texpo=texpo, resol=resol)

In [11]:
rnfrm = mkrdns(ronoi, resol)

In [12]:
eimage = np.multiply(image, queff)   # convert photons to electrons
eimage = np.add(eimage, darkfr)     # add the dark frame
eimage = np.add(eimage, rnfrm)      # add the readnoise

# saturation at full well existence
eimage[eimage > fwell] = fwell

vimage = np.multiply(eimage, ampst)   # convert image to micro-volts/px^2

aduimage = np.multiply(vimage, vtcn)   # convert image to ADU/px^2

# counts rounded-up to the nearest integer
aduimage = np.ceil(aduimage).astype(int)

finim = np.add(aduimage, biasfr)  # add the bias frame

In [13]:
if wpfit:
    for i in range(0, len(biaslist)):
        hdu = fits.PrimaryHDU(biaslist[i])   # make an HDU for the bias image
        hdu.header['IMAGETYP'] = ('BIAS')
        hdu.writeto('Bias_'+str(i+1)+'.fit')   # write the fits

In [14]:
if wpfit:
    for i in range(0, len(darklist)):
        hdu = fits.PrimaryHDU(darklist[i])   # make an HDU for the dark image
        hdu.header['IMAGETYP'] = ('DARK')
        hdu.writeto('Dark_'+str(i+1)+'.fit')   # write the fits

In [15]:
if wpfit:
    hdu = fits.PrimaryHDU(finim)   # make an HDU for the image
    # write in the header that the image is in ADU
    hdu.header['UNIT'] = ('ADU', 'Pixel intensity units')
    hdu.writeto(splitext(filenm)[0] + ('.fit'))   # write the fits